In [89]:
from load_matlab_elmannet import *
import numpy as np
import gym
import time
import os
from PIL import Image

In [90]:
y_min, y_max, x_min, x_max = 25, 195, 20, 140
shape_of_single_frame = (1, (y_max-y_min),(x_max-x_min))

In [120]:
sleep_time = 0.05
model_path = "../2-elman_train/models/play_m45to46_f1to1000_epoch_h200/enduro_elman_1.mat"

is_checkpoint= True
match = 45
data_path = f"../1-generate/data/match_" + str(match) + "/"

In [117]:
ACTIONS = {
        "noop": 0,
        "accelerate": 1,
        "right": 2,
        "left": 3,
        "break": 4,
        "right_break": 5,
        "left_break": 6,
        "right_accelerate": 7,
        "left_accelerate": 8,
}

In [76]:
"""ACTIONS = {
    "right": 2,
    "left": 3,
}"""

'ACTIONS = {\n    "right": 2,\n    "left": 3,\n}'

In [118]:
ACTIONS_LIST = list(ACTIONS.values())

In [37]:
def load_npz(m):
    path = data_path + "npz/"

    actions = np.load(path + 'actions.npz')
    lifes = np.load(path + 'lifes.npz')
    frames = np.load(path + 'frames.npz')
    rewards = np.load(path + 'rewards.npz')

    arr_actions = actions.f.arr_0
    arr_lifes = lifes.f.arr_0
    arr_frames = frames.f.arr_0
    arr_rewards = rewards.f.arr_0

    print("Successfully loaded NPZ.")

    return arr_actions.shape[0], arr_frames, arr_actions, arr_rewards, arr_lifes

In [31]:
num_of_frames, frames, actions, rewards, lifes = load_npz(match)

Successfully loaded NPZ.


In [32]:
frames = frames.reshape(len(frames), -1)
frames = frames[:1000]

In [33]:
frames.shape

(1000, 20400)

In [34]:
def prepare_action_data(action):

    new_action = np.zeros((1, len(ACTIONS)), dtype=int) 

    new_action[0, ACTIONS_LIST.index(action)] = 1

    return new_action

In [35]:
actions

array([0, 0, 0, ..., 3, 0, 0])

In [36]:
targets = [prepare_action_data(actions[i])[0] for i in range(len(actions))]

In [37]:
targets = np.array(targets)

In [121]:
model = LoadElman(model_path, is_checkpoint)

U shape: (200, 20400)
W shape: (200, 200)
V shape: (9, 200)
bh shape: (200,)
bo shape: (9,)


In [10]:
model.reset_prev_s()

print_class = False

for i in range(len(frames)):
    if(print_class):
        print(i+1, np.argmax(targets[i]), "->", np.argmax(model.predict_step(frames[i]/255)))
    else:
        print(i+1, targets[i], "->", model.predict_step(frames[i]/255))

NameError: name 'frames' is not defined

In [40]:
model.reset_prev_s()
predicted = []
correct = 0
wrong = 0
for i in range( len(frames) ):
    
    if np.argmax(targets[i]) == np.argmax(model.predict_step(frames[i]/255)):
        correct += 1
    else:
        wrong +=1

In [41]:
print(correct)
print(wrong)

616
384


In [ ]:
#798/(798+202)

In [88]:
env = gym.make("Enduro-v0")

In [46]:
frame = env.reset()

In [47]:
reward, action, done, info = 0, 0, False, {'ale.lives': 0}

In [48]:
frame.shape

(210, 160, 3)

In [49]:
model.reset_prev_s()

In [126]:
model.reset_prev_s()

env = gym.make("Enduro-v0")
frame = env.reset()
reward, action, done, info = 0, 0, False, {'ale.lives': 0}
env.render()

time.sleep(10)

for _ in range(1000):
    time.sleep(sleep_time)
    env.render()
    
    frame = frame[y_min:y_max, x_min:x_max]

    frame = Image.fromarray(frame)
    frame = frame.convert("L")
    
    frame = np.asarray(frame)
    frame = frame.reshape(20400,)
    frame = frame/255
    
    action = model.predict_step(frame)
    action = list(ACTIONS.values())[np.argmax(action, axis=0)]
    
    print(action)
    
    frame, reward, done, info = env.step(action)

0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
1
0
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
0
0
1
1
1
1
0
1
1
1
1
1
0
0
0
0
0
1
0
0
1
2
0
0
0
0
0
0
0
0
0
0
0
0
0
0
2
0
2
3
3
0
3
3
3
3


KeyboardInterrupt: 